In [22]:
import pandas as pd
import pyspark.sql.functions as F
from tzlocal import get_localzone
local_tz = str(get_localzone())
local_tz

from pyspark.sql.functions import when, lit
from pyspark.sql.types import IntegerType


In [2]:
df_info_session = spark.read.parquet('../output/ifood/ufg_dataset_all/info_session').sort('click_timestamp', 'account_id')
df_info_session.show(20)

+--------------------+--------------------+--------------------+-------------------+---+------------+
|          session_id|          account_id|         merchant_id|    click_timestamp|buy|dt_partition|
+--------------------+--------------------+--------------------+-------------------+---+------------+
|201b8efb-bf7a-42f...|eefd77bc-2288-472...|17b17a05-c383-45f...|2019-06-01 00:00:31|  0|  2019-06-01|
|201b8efb-bf7a-42f...|eefd77bc-2288-472...|83e5a078-2165-4b8...|2019-06-01 00:00:31|  0|  2019-06-01|
|97d7c14f-9bcc-4a7...|74246816-23fe-42f...|a5d66080-c059-4c9...|2019-06-01 00:01:10|  0|  2019-06-01|
|3977c2b6-b8cf-40a...|51d04693-71e0-493...|2cf13ea7-63be-423...|2019-06-01 00:01:17|  0|  2019-06-01|
|3977c2b6-b8cf-40a...|51d04693-71e0-493...|2cf13ea7-63be-423...|2019-06-01 00:01:17|  0|  2019-06-01|
|3977c2b6-b8cf-40a...|51d04693-71e0-493...|2cf13ea7-63be-423...|2019-06-01 00:01:17|  1|  2019-06-01|
|a80ed5b5-d51e-46a...|5c196c3b-050b-42a...|92c5c3a6-40be-4a8...|2019-06-01 00:01:2

In [3]:
# df_info_session2 = spark.read.parquet('../output/ifood/ufg_dataset_all/info_session_merged').sort('click_timestamp', 'account_id')
# df_info_session2.show(20)

In [4]:
# df_info_session.count(), df_info_session2.count()

In [5]:
# df_info_session.intersect(df_info_session2).count()

In [6]:
# df_info_session.unionAll(df_info_session2).except(df_info_session.intersect(df_info_session2)).count()



In [7]:
# df_info_session.count()

In [20]:
df_mearchant = spark.read.csv('../output/ifood/dataset/merchants_for_session_train_5_s=-1.csv',header=True, inferSchema=True)
df_mearchant = df_mearchant.select('merchant_idx', 'merchant_id')
df_mearchant.show(2)

+------------+--------------------+
|merchant_idx|         merchant_id|
+------------+--------------------+
|           0|0050c482-ca8e-4d3...|
|           1|025aa016-d05f-443...|
+------------+--------------------+
only showing top 2 rows



In [9]:
# df_order = spark.read.parquet('../output/ifood/dataset/indexed_orders_test_data_0.10_k=5_s=-1.parquet').sort('click_timestamp')
# df_order.select('session_id', 'account_id', 'click_timestamp', 'merchant_idx').show(2)

In [10]:
# # Create new column in your dataset
# df_order = df_order.withColumn("new_timestamp", F.to_utc_timestamp(df_order.click_timestamp, str(local_tz)))
# df_order.select('session_id', 'account_id', 'click_timestamp', 'new_timestamp', 'merchant_idx').sort('click_timestamp').show(2)

In [11]:
# df_order_pd = pd.read_parquet('../output/ifood/dataset/indexed_orders_test_data_0.10_k=5_s=-1.parquet').sort_values('click_timestamp')
# df_order_pd[['session_id', 'account_id', 'click_timestamp', 'merchant_idx']].head(2)

In [18]:
# /media/workspace/DeepFood/recommendation-system/output/evaluation/EvaluateMostPopularPerUserIfoodModel2/results/MostPopularPerUserIfoodModel_b589e491e0

In [31]:
df_eval = spark.read.csv('/media/workspace/DeepFood/recommendation-system/output/evaluation/EvaluateMostPopularPerUserIfoodModel2/results/MostPopularPerUserIfoodModel_b589e491e0/orders_with_metrics.csv',header=True, inferSchema=True)
df_eval = df_eval.join(df_mearchant, 'merchant_idx', how='inner')

df_rhat_mearchant = df_mearchant.withColumnRenamed("merchant_id", "rhat_merchant_id")\
                    .withColumnRenamed("merchant_idx", "rhat_merchant_idx")

df_eval = df_eval.join(df_rhat_mearchant,'rhat_merchant_idx' , how='inner')\
                    .withColumn("buy", lit(1))

df_eval.limit(10).toPandas()

,rhat_merchant_idx,merchant_idx,_c0,session_id,account_id,dt_partition,account_idx,shift_idx,day_of_week,rhat_scores,...,average_precision,precision_at_1,ndcg_at_5,ndcg_at_10,ndcg_at_15,ndcg_at_20,ndcg_at_50,merchant_id,rhat_merchant_id,buy
0,892,38,0,3a82634c-8544-401a-be71-cd3df9136926,77332d5d-d53a-4f79-82fd-840ba18cfa8b,2019-07-28,54029,3,0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,b6131bcf-54a3-4b51-9afe-205defc99828,a3b7c26d-5eb3-409a-aefa-c50acc7a6222,1
1,43,43,1,89c260ff-2231-4384-9c5e-68989ff7236c,8e4a466f-e8ed-47d3-b5cc-31a6cf9c2a39,2019-07-29,40981,7,1,15.0,...,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,bf97a8ce-be10-4ced-8de9-0ffefe3b688e,bf97a8ce-be10-4ced-8de9-0ffefe3b688e,1
2,418,65,2,78803188-62e4-4417-b370-3b291f1da18a,e32c6f57-cd0e-4417-9a57-c4f59145625b,2019-07-26,56576,6,5,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,9446443b-5c32-4ead-ba69-f4fa024a8f86,25cc4b44-34db-428e-98ec-6a505fd6db76,1
3,738,90,3,be7b3f4c-b66f-4ed0-8fbd-2b1425289e76,e9a83b78-3032-41f2-a333-2bb726f46bc9,2019-07-29,33255,4,1,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,5b2c5581-041d-4eb2-be7d-4a1dcc7899a7,844aad65-fc03-426d-8e8a-0bb95741b54c,1
4,892,91,4,481ea274-b581-445f-896c-f130851482f9,abab9c74-8bff-4cc0-9cd6-6704f77cf6f6,2019-07-26,61168,6,5,0.0,...,0.090909,0.0,0.0,0.000000,0.289065,0.289065,0.289065,67c49040-bf5f-481d-9b00-af6a70c9f66a,a3b7c26d-5eb3-409a-aefa-c50acc7a6222,1
5,631,106,5,439e2f6b-fb56-4a6f-a0cd-7618ef408079,238bbaa4-5660-43b4-8038-186691f4608f,2019-07-24,49327,7,3,2.0,...,0.142857,0.0,0.0,0.356207,0.356207,0.356207,0.356207,bb5505cf-4f5c-41e8-9d5e-e9a16b424ab3,88e9f30b-691c-491c-9c00-c68a8fc0dece,1
6,120,120,6,5b0a7e44-6d48-4eac-b81e-af16d4a7e1eb,7ae7ed1e-53f2-4e4e-a313-6f4b189b48e9,2019-07-26,43851,6,5,3.0,...,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,01728a63-2d91-4e38-b630-8258adb7fddb,01728a63-2d91-4e38-b630-8258adb7fddb,1
7,178,120,7,2f921ca5-5f76-4730-abe1-14ac8b412276,1cb7e3d3-311e-4551-88e1-d77478b3b9f9,2019-07-25,61933,2,4,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,01728a63-2d91-4e38-b630-8258adb7fddb,245f7c05-52d5-4fc3-a709-eec1a87a92aa,1
8,188,188,8,871965eb-54bb-495d-a2f0-231fd9599e4d,be02c785-fa74-4911-b22d-b22983d18d71,2019-07-27,68745,2,6,14.0,...,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,4970c8c8-f97d-4193-942c-677613a6db6c,4970c8c8-f97d-4193-942c-677613a6db6c,1
9,592,318,9,664c8eae-afef-409f-9475-bc5a3184621f,271f2346-5acc-4e18-ba4e-2f046f48bcee,2019-07-25,37467,0,4,4.0,...,0.500000,0.0,1.0,1.000000,1.000000,1.000000,1.000000,852bd163-d47d-4561-a6b4-5150342f8130,5c516c9b-7769-4679-bd42-2849587f1955,1


In [32]:
df_eval.count()

13811

In [35]:
joined = df_info_session.join(
            df_eval.select('session_id', 'account_id', 'merchant_id', 'dt_partition', 'rhat_merchant_id', 'buy'), 
        ['session_id', 'account_id', 'merchant_id', 'dt_partition', 'buy'], how='left').cache()

joined.limit(10).toPandas()

,session_id,account_id,merchant_id,dt_partition,buy,click_timestamp,rhat_merchant_id
0,00078078-ce8e-4b42-b427-219fd31c9175,None,e4f067d0-90bf-4d0f-8498-1c870928b680,2019-07-26,0,2019-07-26 18:18:47,None
1,000c5a06-32fc-4360-9762-78269c01bbb4,f0b5e0bb-9d3a-447a-bd2b-9e5fe6a3b8d3,a7674de6-7657-480f-982f-af071f4e6087,2019-06-05,0,2019-06-05 11:32:36,None
2,0020d0e7-91e0-4f13-8ae4-1cda79418ef4,34c191df-cc3f-4827-a6f4-c5b13f55cf6a,fa480ed1-0347-4337-8da9-6874260345ad,2019-06-19,0,2019-06-19 18:02:04,None
3,0041ea6d-0e1e-40e5-93c6-5ec622c144f6,6fb3b6ed-f7ee-45e3-b50e-555424d72f0a,52f83d6e-795f-42a2-8366-635674f35b37,2019-06-18,0,2019-06-18 15:38:38,None
4,00425831-4ab1-494f-a2e3-24696d1d105f,eea30e4e-4e59-4a25-aec8-1c6e6e9ffd55,c32afbd8-4093-49e6-bc94-ef236528621e,2019-07-22,0,2019-07-22 20:29:55,None
5,0050b9e6-814d-4e50-a9eb-caea89ef2fb8,6b61315c-ffcb-42d0-887a-3bc4632618f2,96b82747-b0e9-409e-a7a5-62edaa791aea,2019-06-03,0,2019-06-03 20:36:50,None
6,0053b10b-bd04-4e8e-8c11-2823fd09ebea,22ff1dee-42ce-43f6-a895-ae79c0cb1da5,8c14939c-68c7-43f8-a743-a07cfcb9ede4,2019-07-19,1,2019-07-19 19:53:53,None
7,0053b10b-bd04-4e8e-8c11-2823fd09ebea,22ff1dee-42ce-43f6-a895-ae79c0cb1da5,8c14939c-68c7-43f8-a743-a07cfcb9ede4,2019-07-19,1,2019-07-19 19:53:53,None
8,0053b10b-bd04-4e8e-8c11-2823fd09ebea,22ff1dee-42ce-43f6-a895-ae79c0cb1da5,8c14939c-68c7-43f8-a743-a07cfcb9ede4,2019-07-19,1,2019-07-19 19:53:53,None
9,00550739-c702-4006-8621-c3f07156c090,46db850d-83fc-48f2-8e95-2aa60b15bf64,5f4f09ed-3ef8-4d5d-9772-e8cf4322cbe5,2019-07-28,0,2019-07-28 19:22:51,None


In [36]:
joined.filter(joined.rhat_merchant_id.isNotNull()).count()

58860

In [37]:
joined.printSchema()

root
 |-- session_id: string (nullable = true)
 |-- account_id: string (nullable = true)
 |-- merchant_id: string (nullable = true)
 |-- dt_partition: string (nullable = true)
 |-- buy: integer (nullable = true)
 |-- click_timestamp: timestamp (nullable = true)
 |-- rhat_merchant_id: string (nullable = true)



In [97]:

df = joined
#df = df.withColumn('acc', joined.merchant_id == joined.rhat_merchant_id).cache()
df = df.withColumn('buy_2', when(df.rhat_merchant_id.isNull(), df.buy).otherwise((joined.merchant_id == joined.rhat_merchant_id).cast(IntegerType())))
df = df.withColumn('merchant_id_2', when(df.rhat_merchant_id.isNull(), df.merchant_id).otherwise(df.rhat_merchant_id))

df = df.limit(5000).toPandas()
df.head()#.show(10)

,session_id,account_id,click_timestamp,buy,merchant_id,dt_partition,rhat_merchant_id,buy_2,merchant_id_2
0,0006707d-028e-450f-8b5f-82ca6afb6004,4d44c19a-0173-4d29-b0b1-a3ab3f18dd61,2019-06-23 08:50:54,0,47e4c365-5047-49a4-8f7c-37ffa2f15579,2019-06-23,None,0,47e4c365-5047-49a4-8f7c-37ffa2f15579
1,0006ab6a-22f6-48ab-9fd2-4b4f12eefd61,8be8733e-6171-4be5-9645-c29bb945fd13,2019-06-18 11:50:56,0,3cc76a2d-794a-4517-8ade-23ec49e6a381,2019-06-18,None,0,3cc76a2d-794a-4517-8ade-23ec49e6a381
2,0009f616-ff45-41be-9d3c-99ee55744b1a,30266171-963e-44a1-8861-3e281eb54e8e,2019-07-17 12:33:14,0,25036a46-d535-427f-b09a-c254e58429eb,2019-07-17,None,0,25036a46-d535-427f-b09a-c254e58429eb
3,0009f616-ff45-41be-9d3c-99ee55744b1a,30266171-963e-44a1-8861-3e281eb54e8e,2019-07-17 12:33:14,0,25036a46-d535-427f-b09a-c254e58429eb,2019-07-17,None,0,25036a46-d535-427f-b09a-c254e58429eb
4,000dcacb-bec0-4a95-8fda-777b855afb39,ad8fea0c-1a6a-4d78-a8da-a3f9807abefc,2019-06-01 01:02:47,0,d5812250-d991-44a0-a05d-98ae14b0be95,2019-06-01,None,0,d5812250-d991-44a0-a05d-98ae14b0be95


In [106]:
df[(df.buy_2 == 1) & (df.buy == 1) & (~df.rhat_merchant_id.isna())]

,session_id,account_id,click_timestamp,buy,merchant_id,dt_partition,rhat_merchant_id,buy_2,merchant_id_2
364,029f862e-9a7e-4d97-87d2-006c40501891,2d6f93b9-10f0-4e7e-a476-75ab3e33b384,2019-07-27 15:23:01,1,bdc1a67c-f0be-40a5-8594-242c419e74ce,2019-07-27,bdc1a67c-f0be-40a5-8594-242c419e74ce,1,bdc1a67c-f0be-40a5-8594-242c419e74ce
1339,09b8f149-8923-4a7d-82fa-7e863f32f8bd,56308fe2-fe1d-4bb3-89f4-bc263a18687a,2019-07-25 14:05:34,1,bf97a8ce-be10-4ced-8de9-0ffefe3b688e,2019-07-25,bf97a8ce-be10-4ced-8de9-0ffefe3b688e,1,bf97a8ce-be10-4ced-8de9-0ffefe3b688e
2939,153c373b-e8d2-43a5-bcaf-fad45d94d067,6122925e-916a-4c93-b46b-242fcb3e6437,2019-07-30 20:32:48,1,e99d0cbe-c311-49fa-84d6-9e8406ef184a,2019-07-30,e99d0cbe-c311-49fa-84d6-9e8406ef184a,1,e99d0cbe-c311-49fa-84d6-9e8406ef184a
3465,190e754b-00be-4f0a-9549-b94d0e383a9f,c6c16677-c9b9-422a-ac26-a436e48fb7a5,2019-07-31 12:45:48,1,4fb45c04-0ffa-4a2a-9f4f-9eda441b0992,2019-07-31,4fb45c04-0ffa-4a2a-9f4f-9eda441b0992,1,4fb45c04-0ffa-4a2a-9f4f-9eda441b0992


In [ ]:
.withColumn('Id_New',when(df.Rank <= 5,df.Id).otherwise('other'))\


In [43]:
joined.count()

7255085

In [45]:
joined.filter(joined.rhat_merchant_id.isNotNull()).count()

68311

In [ ]:
joined.count()

In [107]:
joined.columns

['session_id',
 'account_id',
 'click_timestamp',
 'buy',
 'merchant_id',
 'dt_partition',
 'rhat_merchant_id']

In [23]:
df_info_session.filter(df_info_session.session_id == "511695b1-1a84-482e-af81-86f3cb05bd9b").sort('click_timestamp').show(50)

+--------------------+--------------------+--------------------+-------------------+---+------------+
|          session_id|          account_id|         merchant_id|    click_timestamp|buy|dt_partition|
+--------------------+--------------------+--------------------+-------------------+---+------------+
|511695b1-1a84-482...|2659f5e6-4e39-4a9...|01728dcb-857d-4a2...|2019-07-30 17:59:43|  1|  2019-07-30|
|511695b1-1a84-482...|2659f5e6-4e39-4a9...|01728dcb-857d-4a2...|2019-07-30 17:59:43|  1|  2019-07-30|
|511695b1-1a84-482...|2659f5e6-4e39-4a9...|01728dcb-857d-4a2...|2019-07-30 17:59:58|  1|  2019-07-30|
|511695b1-1a84-482...|2659f5e6-4e39-4a9...|01728dcb-857d-4a2...|2019-07-30 17:59:58|  1|  2019-07-30|
|511695b1-1a84-482...|2659f5e6-4e39-4a9...|01728dcb-857d-4a2...|2019-07-30 18:00:20|  1|  2019-07-30|
|511695b1-1a84-482...|2659f5e6-4e39-4a9...|01728dcb-857d-4a2...|2019-07-30 18:00:20|  1|  2019-07-30|
|511695b1-1a84-482...|2659f5e6-4e39-4a9...|01728dcb-857d-4a2...|2019-07-30 18:00:2

In [24]:
df_eval.filter(df_eval.session_id == "511695b1-1a84-482e-af81-86f3cb05bd9b").select('account_id', 'click_timestamp', 'click_timestamp_utc').sort('click_timestamp').show(50)

+--------------------+-------------------+-------------------+
|          account_id|    click_timestamp|click_timestamp_utc|
+--------------------+-------------------+-------------------+
|2659f5e6-4e39-4a9...|2019-07-30 20:59:43|2019-07-30 17:59:43|
|2659f5e6-4e39-4a9...|2019-07-30 20:59:58|2019-07-30 17:59:58|
|2659f5e6-4e39-4a9...|2019-07-30 21:00:20|2019-07-30 18:00:20|
|2659f5e6-4e39-4a9...|2019-07-30 21:00:29|2019-07-30 18:00:29|
|2659f5e6-4e39-4a9...|2019-07-30 21:03:50|2019-07-30 18:03:50|
|2659f5e6-4e39-4a9...|2019-07-30 21:04:49|2019-07-30 18:04:49|
|2659f5e6-4e39-4a9...|2019-07-30 21:05:35|2019-07-30 18:05:35|
|2659f5e6-4e39-4a9...|2019-07-30 21:05:46|2019-07-30 18:05:46|
|2659f5e6-4e39-4a9...|2019-07-30 21:05:51|2019-07-30 18:05:51|
|2659f5e6-4e39-4a9...|2019-07-30 21:06:05|2019-07-30 18:06:05|
|2659f5e6-4e39-4a9...|2019-07-30 21:06:56|2019-07-30 18:06:56|
|2659f5e6-4e39-4a9...|2019-07-30 21:08:06|2019-07-30 18:08:06|
+--------------------+-------------------+-------------

In [25]:
df_order.filter(df_order.session_id == "511695b1-1a84-482e-af81-86f3cb05bd9b").select('account_id', 'click_timestamp').sort('click_timestamp').show(50)

+--------------------+-------------------+
|          account_id|    click_timestamp|
+--------------------+-------------------+
|2659f5e6-4e39-4a9...|2019-07-30 17:59:43|
|2659f5e6-4e39-4a9...|2019-07-30 17:59:58|
|2659f5e6-4e39-4a9...|2019-07-30 18:00:20|
|2659f5e6-4e39-4a9...|2019-07-30 18:00:29|
|2659f5e6-4e39-4a9...|2019-07-30 18:03:50|
|2659f5e6-4e39-4a9...|2019-07-30 18:04:49|
|2659f5e6-4e39-4a9...|2019-07-30 18:05:35|
|2659f5e6-4e39-4a9...|2019-07-30 18:05:46|
|2659f5e6-4e39-4a9...|2019-07-30 18:05:51|
|2659f5e6-4e39-4a9...|2019-07-30 18:06:05|
|2659f5e6-4e39-4a9...|2019-07-30 18:06:56|
|2659f5e6-4e39-4a9...|2019-07-30 18:08:06|
+--------------------+-------------------+



In [26]:
_df_eval = pd.read_csv('../output/evaluation/EvaluateAutoEncoderIfoodModel/results/VariationalAutoEncoderTraining_selu____500_fc62ac744a_6534ac1232/orders_with_metrics.csv')
_df_eval.head(2)
_df_eval[_df_eval.session_id == "511695b1-1a84-482e-af81-86f3cb05bd9b"].sort_values('click_timestamp')

,Unnamed: 0,session_id,account_id,click_timestamp,account_idx,merchant_idx,rhat_merchant_idx,shift_idx,day_of_week,rhat_scores,...,count_buys,rhat_rewards,rewards,average_precision,precision_at_1,ndcg_at_5,ndcg_at_10,ndcg_at_15,ndcg_at_20,ndcg_at_50
16439,16439,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 20:59:43,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25519,25519,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 20:59:58,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29045,29045,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 21:00:20,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6867,6867,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 21:00:29,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9243,9243,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 21:03:50,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12828,12828,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 21:04:49,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5595,5595,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 21:05:35,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 21:05:46,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15103,15103,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 21:05:51,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8467,8467,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 21:06:05,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df_eval.columns

Index(['Unnamed: 0', 'session_id', 'account_idx', 'merchant_idx',
       'rhat_merchant_idx', 'shift_idx', 'day_of_week', 'rhat_scores', 'ps',
       'ps_eval', 'count_visits', 'count_buys', 'rhat_rewards', 'rewards',
       'average_precision', 'precision_at_1', 'ndcg_at_5', 'ndcg_at_10',
       'ndcg_at_15', 'ndcg_at_20', 'ndcg_at_50'],
      dtype='object')

In [6]:
df_eval.shape

(34940, 23)

In [11]:
df_eval.groupby(['session_id', 'account_id', 'click_timestamp']).count().shape

(34940, 20)